In [36]:
# Import statements. 
import gensim, logging
import pandas as pd
import numpy as np
import ast
from nltk import download, FreqDist, WordNetLemmatizer
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.metrics import accuracy_score
 
# Build sentences from review data.
sentences = []
word_list = []
word_set = []
data = pd.read_csv('https://raw.githubusercontent.com/csbanon/bert-product-rating-predictor/master/clustering_data/tokens_predicted_5.csv')
print(data.axes)
for comment in data['review']:
  temp_sentence = ast.literal_eval(comment)
  sentences.append(temp_sentence)
  for word in temp_sentence:
    word_list.append(word)

frequency_dist = FreqDist(word_list)
word_set = list(set([word for word in word_list if word !="" and not word.isnumeric()]))

[RangeIndex(start=0, stop=29340, step=1), Index(['Unnamed: 0', 'review', 'prediction'], dtype='object')]


In [37]:
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2020-11-25 16:23:21,918 : INFO : collecting all words and their counts
2020-11-25 16:23:21,919 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-11-25 16:23:21,976 : INFO : PROGRESS: at sentence #10000, processed 236464 words, keeping 12522 word types
2020-11-25 16:23:22,026 : INFO : PROGRESS: at sentence #20000, processed 470185 words, keeping 17241 word types
2020-11-25 16:23:22,077 : INFO : collected 20702 word types from a corpus of 692673 raw words and 29340 sentences
2020-11-25 16:23:22,078 : INFO : Loading a fresh vocabulary
2020-11-25 16:23:22,119 : INFO : effective_min_count=1 retains 20702 unique words (100% of original 20702, drops 0)
2020-11-25 16:23:22,120 : INFO : effective_min_count=1 leaves 692673 word corpus (100% of original 692673, drops 0)
2020-11-25 16:23:22,194 : INFO : deleting the raw counts dictionary of 20702 items
2020-11-25 16:23:22,196 : INFO : sample=0.001 downsamples 50 most-common words
2020-11-25 16:23:22,196 : INFO : downs

In [38]:
# Testing
top_words = frequency_dist.get('computer')
print(top_words)
print(model.wv['computer'])

621
[-0.31534117  0.00725356 -0.11354424 -0.30353072 -0.05212222  1.014518
 -0.36917853 -0.68613374  0.4344511  -0.05051461 -0.5368282  -0.31623566
  0.2487738   0.02004026  0.0284112   0.8526692   0.27090377  0.14540307
 -0.15304027  0.1427102  -0.30992174 -1.060559    1.0983423  -0.6307261
 -0.47346342 -0.2870233  -0.02979285  0.22083788  0.05377037  0.54229134
 -0.4828033  -0.03148756  0.5196573   0.02854848 -0.82863605 -0.41284472
  0.14356743  1.0117517   0.06946371 -0.01912882  0.00173389 -0.5376283
 -0.2605944  -0.45136312 -0.92593753  0.14019379  0.10345034  0.30725998
  0.46850967 -0.16337895 -0.10074654 -0.5862097  -0.26552865 -0.16341934
 -1.2119588   0.42923656  0.08763079 -0.86574125  0.21358761 -0.11940615
  0.12111821 -0.22120863  0.7133103   1.0570287   0.5075445  -0.23019737
 -0.5138585   0.85307616  1.1395689   0.50121224 -0.64492726  0.73598444
  0.5910498  -0.23985393 -0.35875228  0.12158713 -0.03731176  0.04585237
  0.29201648 -0.24007073  0.7014852  -0.94085336  0

In [39]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(model.wv.vectors)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [40]:
labels = kmeans.labels_
centers = kmeans.cluster_centers_
clusters = []
clusters_count = []

# Calculate Euclidian distance with vectorized words. 
def calc_distance(a, b):
  total = 0
  for i in range(0, a.size):
    total += (a[i] - b[i]) ** 2
  return np.sqrt(total)

# Create a list of each value for each cluster. 
# This will make later operations much more efficient.
for i in range(0, len(centers)):
  cluster_list = []
  cluster_list_count = []
  for x in range(0, len(labels)):
    if labels[x] == i:
      cluster_list.append(word_set[x])
      cluster_list_count.append(frequency_dist.get(word_set[x]))
  clusters.append(cluster_list)
  clusters_count.append(np.array(cluster_list_count))

# Print words of each cluster.
for i in range(0, len(clusters)):
  print("\nWords in cluster ", i+1)
  top_5_index = clusters_count[i].argsort()[-5:][::-1]
  top_5 = []
  top_5_count = []
  for index in top_5_index:
    top_5.append(clusters[i][index])
    top_5_count.append(frequency_dist.get(clusters[i][index]))
  print(top_5)
  print(top_5_count)

# Find the center word for each cluster. 
center_words = []
center_word_distances = []

for i in range(0, 5):
  distances = []
  for x in range(0, len(word_set)):
    distances.append(calc_distance(centers[i], model.wv[word_set[x]]))
  mins = min(distances)
  mins_index = distances.index(mins)
  center_word_distances.append(mins)
  center_words.append(word_set[mins_index])

print(center_words)


Words in cluster  1
['not', 'work', 'sound', 'one', 'good']
[9104, 7559, 6387, 6060, 5891]

Words in cluster  2
['alexa', 'put', 'show', 'started', 'cannot']
[1888, 1035, 990, 406, 392]

Words in cluster  3
['great', 'no', 'phone', 'device', 'even']
[9451, 4243, 3885, 2558, 2283]

Words in cluster  4
['camera', 'paper', 'different', 'reason', 'whole']
[3165, 1098, 917, 494, 239]

Words in cluster  5
['would', 'local', 'generation', 'status', 'motherboard']
[4105, 285, 129, 88, 34]
['aos', 'honestly', 'challenge', 'basically', 'excelente']


In [41]:
print(center_words)
print(center_word_distances)

['aos', 'honestly', 'challenge', 'basically', 'excelente']
[0.0349795329042444, 1.760378641942225, 0.462598645284901, 1.7748286672736797, 0.7348057779545784]
